Ensenmble

 - 여러 개의 모델을 결합하여 단일 모델보다 더 나은 예측 성능을 얻는 기법

 - 보팅 (Voting) : hard voting (여러 모델의 결과 중 다수결) / sort voting (예측 확률을 평균 -> predict_proba 를 지원하는 모델만)

 - 베깅 (Bagging) : 같은 알고리즘 모델을 병렬 결합 (Bootstrapping aggregation)

 - 부스팅 (Boosting) : 여러개의 모델으로 순차적으로 학습하여 결과에 대해 가중치를 부여하고 반복 학습 (오답에 높은 가중치 -> 오답율 정답으로 맞추기 위해 더 집중)

 - 스태킹 (Stacking) : 여러개의 다른 모델을 학습하여 예측한 결과를 다시 feature로 사용. 해당 feature들로 최종 모델이 학습하여 결과

 대표적인 앙상블 알고리즘

 - 랜덤 포레스트 (Random Forest) :  decision tree 가 여러개 -> Bagging

 - 그래디언트 부스팅 (Gradient Boosting) :  이진 트리의 오차를 보완하는 방식으로 순차적 학습 -> Boosting

 - XGBoost

 - LightGBM


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd

In [2]:
mr = pd.read_csv("../datasets/mushrooms.csv")
mr

type cap_shape cap_surface cap_color bruises odor gill_attachment  \
0       p         x           s         n       t    p               f   
1       e         x           s         y       t    a               f   
2       e         b           s         w       t    l               f   
3       p         x           y         w       t    p               f   
4       e         x           s         g       f    n               f   
...   ...       ...         ...       ...     ...  ...             ...   
8119    e         k           s         n       f    n               a   
8120    e         x           s         n       f    n               a   
8121    e         f           s         n       f    n               a   
8122    p         k           y         n       f    y               f   
8123    e         x           s         n       f    n               a   

     gill_spacing gill_size gill_color  ... stalk_surface_below_ring  \
0               c         n          k  ...                        s   
1               c         b          k  ...                        s   
2               c         b          n  ...                        s   
3               c         n          n  ...                        s   
4               w         b          k  ...                        s   
...           ...       ...        ...  ...                      ...   
8119            c         b          y  ...                        s   
8120            c         b          y  ...                        s   
8121            c         b          n  ...                        s   
8122            c         n          b  ...                        k   
8123            c         b          y  ...                        s   

     stalk_color_above_ring stalk_color_below_ring veil_type veil_color  \
0                         w                      w         p          w   
1                         w                      w         p          w   
2                         w                      w         p          w   
3                         w                      w         p          w   
4                         w                      w         p          w   
...                     ...                    ...       ...        ...   
8119                      o                      o         p          o   
8120                      o                      o         p          n   
8121                      o                      o         p          o   
8122                      w                      w         p          w   
8123                      o                      o         p          o   

     ring_number ring_type spore_print_color population habitat  
0              o         p                 k          s       u  
1              o         p                 n          n       g  
2              o         p                 n          n       m  
3              o         p                 k          s       u  
4              o         e                 n          a       g  
...          ...       ...               ...        ...     ...  
8119           o         p                 b          c       l  
8120           o         p                 b          v       l  
8121           o         p                 b          c       l  
8122           o         e                 w          v       l  
8123           o         p                 o          c       l  

[8124 rows x 23 columns]

In [3]:
mr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   type                      8124 non-null   object
 1   cap_shape                 8124 non-null   object
 2   cap_surface               8124 non-null   object
 3   cap_color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill_attachment           8124 non-null   object
 7   gill_spacing              8124 non-null   object
 8   gill_size                 8124 non-null   object
 9   gill_color                8124 non-null   object
 10  stalk_shape               8124 non-null   object
 11  stalk_root                8124 non-null   object
 12  stalk_surface_above_ring  8124 non-null   object
 13  stalk_surface_below_ring  8124 non-null   object
 14  stalk_color_above_ring  

In [ ]:

# e(edible 식용) / p (poisonous 독버섯)
mr.iloc[:, 0].value_counts()

type
e    4208
p    3916
Name: count, dtype: int64

In [ ]:
# data 가 모두 문자열로 되어 있어서, 숫자로 변환하고 싶다.
# ord : 문자열 -> ascii code (0 ~ 127)
print(ord("a"))

97


In [6]:
x = []
y = []

for idx, row in mr.iterrows():
    y.append(row.iloc[0])

    row_data =[]
    for data in row.iloc[1:]:
        row_data.append(ord(data))

    x.append(row_data)

print(x[:5])
print(y[:5])

[[120, 115, 110, 116, 112, 102, 99, 110, 107, 101, 101, 115, 115, 119, 119, 112, 119, 111, 112, 107, 115, 117], [120, 115, 121, 116, 97, 102, 99, 98, 107, 101, 99, 115, 115, 119, 119, 112, 119, 111, 112, 110, 110, 103], [98, 115, 119, 116, 108, 102, 99, 98, 110, 101, 99, 115, 115, 119, 119, 112, 119, 111, 112, 110, 110, 109], [120, 121, 119, 116, 112, 102, 99, 110, 110, 101, 101, 115, 115, 119, 119, 112, 119, 111, 112, 107, 115, 117], [120, 115, 103, 102, 110, 102, 119, 98, 107, 116, 101, 115, 115, 119, 119, 112, 119, 111, 101, 110, 97, 103]]
['p', 'e', 'e', 'p', 'e']


In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [ ]:
# RandomForestClassifier : binary classification (이진 분류, 0 or 1) :  성능이 낮은 것을 가져다가 성능을 높여줌 (기대값이 커지기 때문에)

# n_estimators : 결정트리의 갯수, 많을수록 좋은 성능을 기대할 수 있지만 속도가 느려짐 
# max_depth : 결정트리의 최대 깊이
# n_jobs : 병렬처리 할 cpu 갯수 (-1 : 전체 cpu 모두 사용)

model = RandomForestClassifier(n_estimators=200, max_depth=5,n_jobs=-1)

In [9]:
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=200, n_jobs=-1)

In [10]:
predict = model.predict(x_test)

In [11]:
accuracy_score(y_test, predict)

0.9864615384615385

In [12]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           e       0.97      1.00      0.99       820
           p       1.00      0.97      0.99       805

    accuracy                           0.99      1625
   macro avg       0.99      0.99      0.99      1625
weighted avg       0.99      0.99      0.99      1625



In [14]:
model.estimators_

[DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1751554657),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=85207744),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1802129947),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1592494491),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=304710590),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1421464298),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1207860205),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=744692370),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=283653920),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1262456274),
 DecisionTreeClassifi